In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCN8CPzwkYiDVLZlgD4JQgJQ/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 272


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

8:16
2:01
9:08
7:38
5:01
3:05
6:56
5:54
7:39
3:02
11:00
6:20
9:48
9:46
0:38
5:58
6:21
3:22
6:49
4:19
6:26
9:53
3:00
7:53
7:28
5:31
9:35
3:29
6:25
5:00
8:25
5:00
1:07
7:04
8:12
2:38
4:23
5:36
4:19
8:18
7:15
8:23
6:32
7:21
6:07
5:45
3:41
2:12
2:25
4:57
6:08
5:04
5:21
7:32
5:22
7:07
7:33
5:00
4:45
6:12
4:36
11:21
4:29
1:33
6:29
4:55
9:10
6:45
4:37
6:14
8:52
6:38
5:06
13:57
3:16
6:36
9:20
5:35
2:00
7:53
3:02
6:07
7:27
7:07
6:23
3:47
7:11
6:19
6:07
7:03
6:57
12:30
4:56
6:42
3:40
7:12
3:24
6:32
1:32
7:06
7:59
5:50
6:06
5:39
9:04
5:30
2:55
8:21
8:15
5:50
7:00
6:45
7:57
8:26
5:12
3:54
6:05
2:08
2:42
6:15
6:13
6:35
8:01
5:18
5:46
4:33
4:16
5:41
7:05
6:20
6:50
3:31
5:12
7:59
6:25
8:49
6:32
0:39
4:32
6:03
5:02
7:12
6:57
11:40
6:34
7:13
1:08
8:03
7:21
4:41
6:25
1:27
9:04
8:18
6:13
6:51
7:57
8:06
3:55
8:09
7:49
7:38
6:51
5:32
0:14
6:38
8:50
14:13
6:29
8:22
6:44
4:52
5:49
5:51
5:36
9:19
10:19
4:37
7:20
5:05
7:59
7:35
0:29
6:01
5:42
6:59
6:33
6:23
7:40
7:41
8:54
4:59
11:53
5:50
6:31
4:35
8:08
3:09
10

In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [5]:
for i in range(len(video_list2)):
    #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
    thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
    title = video_list2[i].find('a',{'id':'video-title'}).text # 2 
    
    # 여기서부터 영상 클릭해서 들어간 페이지
    click_url = park_url[i]
    browser.get(click_url)
    time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
    # body = browser.find_element_by_tag_name('body')
    browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
    time.sleep(3)
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
   
    # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
    category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
    view_count = html.find('span', {'class': 'view-count'}).text # 4
    
    # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
    # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
    likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
    'aria-label':re.compile('좋아요')}).text+'개'
    dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
    'aria-label':re.compile('싫어요')}).text+'개'
    upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
    
    print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    thumbnail_list.append(thumbnail)
    title_list.append(title)
    category_list.append(category)
    view_count_list.append(view_count)
    likes_count_list.append(likes_count)
    dislikes_count_list.append(dislikes_count)
    upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/1LW33ylk8wM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAKMobJzYv2PTYQsoZ6vJKWonw7lg 제목: 박막례 간장게장 만들기 [박막례 할머니] 카테고리 코미디 조회수 조회수 612,689회 좋아요수 1.9만개 싫어요수 223개 게시일 2020. 1. 5.
1 . 썸네일 https://i.ytimg.com/vi/BnmL_KbnStE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAtKUvlAfw_Tr8TbtOVezuPGN3qyg 제목: 핫팩 사세요~ 호~ HOT핫HOT핫팩이 왔어요~❤️ [박막례 할머니] 카테고리 코미디 조회수 조회수 126,426회 좋아요수 7.4천개 싫어요수 31개 게시일 2020. 1. 3.
2 . 썸네일 https://i.ytimg.com/vi/nbQQauKcHmk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDVsZI6akM_iSkowKHTchRmfaME5w 제목: 그 연예대상 시상식 브이로그!!!  [박막례 할머니] 카테고리 코미디 조회수 조회수 538,442회 좋아요수 2.4만개 싫어요수 141개 게시일 2019. 12. 29.
3 . 썸네일 https://i.ytimg.com/vi/NrTC1wqiGVQ/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAyCTp96M5hXLjXPM_UbJUCUw2xpQ 제목: 크리스마스 홈파티 인 파리 [박막례 할머니] 카테고리 코미디 조회수 조회수 246,666회 좋아요수 1만개 싫어요수 73개 게시일 2019. 12. 24.
4 . 썸네일 https://i.ytimg.com/vi/8UjKbZQH

34 . 썸네일 https://i.ytimg.com/vi/2dgxsgw39d8/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLA6XIgDc3QgPnCUqPs-sbeOcji1BQ 제목: (Eng)설탕폭탄! 흑당버블티 만들기 [박막례 할머니] 카테고리 코미디 조회수 조회수 1,416,848회 좋아요수 2.6만개 싫어요수 581개 게시일 2019. 9. 1.
35 . 썸네일 https://i.ytimg.com/vi/B_h4XdiqxP0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCNtTNraHkSXdRCzpPg5ohpKWmmEA 제목: 나의 아주 멋진 영정사진 [박막례 할머니] 카테고리 코미디 조회수 조회수 1,187,817회 좋아요수 3.5만개 싫어요수 264개 게시일 2019. 8. 27.
36 . 썸네일 https://i.ytimg.com/vi/U8h5Rv64XLM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD9Cuv371zROog2BQcnogOm_74WyA 제목: (Eng)킬러그랜마님이 접속하셨습니다. GTA5 집 없는 설움 [박막례 할머니] 카테고리 코미디 조회수 조회수 414,255회 좋아요수 1.1만개 싫어요수 140개 게시일 2019. 8. 20.
37 . 썸네일 https://i.ytimg.com/vi/mTxLVsl50xY/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC5WcV7w20vBqkE5Q9VrKHXL98TuA 제목: (Eng)박막례 근황, 일상 브이로그 [박막례 할머니] 카테고리 코미디 조회수 조회수 760,750회 좋아요수 2.1만개 싫어요수 230개 게시일 2019. 8. 16.
38 . 썸네일 https

67 . 썸네일 https://i.ytimg.com/vi/BdJDzRuyAT0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCOeTpJLDB4bzxaMoio1q0QlKOxHA 제목: (Eng)누가 탕후루를 그렇게 먹어요.. 이상한 탕후루 만들기 [박막례 할머니] 카테고리 코미디 조회수 조회수 841,834회 좋아요수 1.5만개 싫어요수 220개 게시일 2019. 5. 15.
68 . 썸네일 https://i.ytimg.com/vi/gdgw_La_Sbc/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCZ12brNHEfamu8DYYIYmqhNm9bOw 제목: (Eng)블루보틀 본고장!! 샌프란시스코에서 마셔본 리뷰 [박막례 할머니] 카테고리 코미디 조회수 조회수 402,748회 좋아요수 9.7천개 싫어요수 89개 게시일 2019. 5. 12.
69 . 썸네일 https://i.ytimg.com/vi/WG9d3Toe7Dk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBR2mziuolRlyOrgTbv1o9biOFWNg 제목: (Eng)50만원어치 택배 언박싱 (식품이 절반..) [박막례 할머니] 카테고리 코미디 조회수 조회수 1,225,579회 좋아요수 2만개 싫어요수 338개 게시일 2019. 5. 8.
70 . 썸네일 https://i.ytimg.com/vi/thw54v0v12Y/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBes_HGYedJGtDTg6jysp4UALf5WA 제목: (Eng)최신곡 들리는대로 부르기ㅋㅋㅋㅋㅋㅋㅋㅋ [박막례 할머니] 카테고리 코미디 조회수 조회수 1,260,149회 좋아요수 3.2만개 싫어요수 536개 게

100 . 썸네일 https://i.ytimg.com/vi/Os-XpPfTYg4/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLADr07oTG2cXicXq-LdVKnYsnpiRA 제목: 완전 쫀득쫀득 얼큰한 김치수제비!! 후다닥 해먹기 [박막례 할머니] 카테고리 Sangmi Goh 조회수 조회수 3,216,474회 좋아요수 4.3만개 싫어요수 866개 게시일 2019. 1. 27.
101 . 썸네일 https://i.ytimg.com/vi/jDKN0Qk1y08/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCMK-0i5rB2BHy-zuZ-ryOISy7ujQ 제목: (Eng)촉으로 물건 맞추는 막례쓰 [박막례 할머니] 카테고리 코미디 조회수 조회수 407,759회 좋아요수 1만개 싫어요수 73개 게시일 2019. 1. 22.
102 . 썸네일 https://i.ytimg.com/vi/xykrAYi-Ry4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCu_L2_KVAuDg8guqRqelayNkJp2w 제목: (Eng)짧은 일상, 누워서 드라마 보기 [박막례 할머니] 카테고리 코미디 조회수 조회수 839,474회 좋아요수 1.5만개 싫어요수 177개 게시일 2019. 1. 20.
103 . 썸네일 https://i.ytimg.com/vi/8cNVSv1YmKI/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBnr1vSLERJ2GPYN8DA3gDGv_lHrQ 제목: (Eng)서로 서운한거 말해보자 (feat.수영고모) [박막례 할머니] 카테고리 코미디 조회수 조회수 1,637,135회 좋아요수 2.3만개 싫어요수 340개 게시일 2019. 1.

133 . 썸네일 https://i.ytimg.com/vi/I65rxEnzxX4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAC09DCCprL5KvuHpB8rk0m2AegIg 제목: (Eng)로마에서 가방 통째로 도둑맞다.. [박막례 할머니] 카테고리 코미디 조회수 조회수 693,371회 좋아요수 1.1만개 싫어요수 130개 게시일 2018. 9. 24.
134 . 썸네일 https://i.ytimg.com/vi/u5C9efMiI3A/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAe9AZG56qhRQTmNS_m5flbgamUGg 제목: 막례를 질리게 만든 중국당면 시식기 [박막례 할머니] 카테고리 코미디 조회수 조회수 2,102,602회 좋아요수 2.4만개 싫어요수 423개 게시일 2018. 9. 21.
135 . 썸네일 https://i.ytimg.com/vi/I_uFgLWYvxE/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDzy6EzMFCZ5sPY_QdQiJX2MIyk9A 제목: (Eng)누가 막례를 독일로 불렀을까? [박막례 할머니] 카테고리 코미디 조회수 조회수 598,147회 좋아요수 1.3만개 싫어요수 194개 게시일 2018. 9. 9.
136 . 썸네일 https://i.ytimg.com/vi/6PWVpVcD98s/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDBmDbxctcnu6gKAwbGuI4NSoJjgg 제목: (Eng)2분컷, 죠리퐁 빙수 만들기 [박막례 할머니] 카테고리 코미디 조회수 조회수 881,977회 좋아요수 1.5만개 싫어요수 185개 게시일 2018. 9. 6.
137 . 썸네일 https:/

166 . 썸네일 https://i.ytimg.com/vi/qj4MBiD8k9E/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDv8Apwh3XMFYOjFR36DFEpWOjLSQ 제목: 드라마 덕질 끝에.. 진짜 권상우 만나다! [박막례 할머니] 카테고리 코미디 조회수 조회수 1,187,329회 좋아요수 1.9만개 싫어요수 296개 게시일 2018. 5. 31.
167 . 썸네일 https://i.ytimg.com/vi/VpAtI2rL1cU/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDwycOWtfyuISqFo8mGA5mtiHCFTw 제목: 진짜 구글을 뒤집어 놓으셨다..! (유튜브 CEO 찾기) [박막례 할머니] 카테고리 코미디 조회수 조회수 1,521,653회 좋아요수 3.5만개 싫어요수 479개 게시일 2018. 5. 26.
168 . 썸네일 https://i.ytimg.com/vi/vzx0QKKqBbc/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAFoADIy2Vhh-uI8Pr8qs7ZgxOWEg 제목: 막례의 스탠딩 코미디쑈 (싸이월드 디스하기) [박막례 할머니] 카테고리 코미디 조회수 조회수 205,061회 좋아요수 4.6천개 싫어요수 116개 게시일 2018. 5. 22.
169 . 썸네일 https://i.ytimg.com/vi/L_ySSTvZtlY/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDPt8jYMcHKxLKtvCQlidSSd2Y3cw 제목: 밥상머리 없는 강아지와 피자 겸상하기 [박막례 할머니] 카테고리 동연 조회수 조회수 729,205회 좋아요수 1.1만개 싫어요수 168개 게시일 2018. 5. 14.

199 . 썸네일 https://i.ytimg.com/vi/yMahcF6Fopk/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCnsH6nI10PHBvFU7TLD0lxtFB_AQ 제목: 오케이 ~ 편한테 신발 선물 받은 막례할머니 [박막례 할머니] 카테고리 코미디 조회수 조회수 226,843회 좋아요수 7.8천개 싫어요수 37개 게시일 2017. 12. 20.
200 . 썸네일 https://i.ytimg.com/vi/XaCrDESZGDA/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC_D1SDyvPSOSyvfrkS0kAUb-crVQ 제목: 가난해도 강력했던 1970년대 메이크업  [박막례 할머니] 카테고리 코미디 조회수 조회수 1,894,565회 좋아요수 3.2만개 싫어요수 734개 게시일 2017. 12. 13.
201 . 썸네일 https://i.ytimg.com/vi/RqT11HD8uf8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDwROVDI7aMxbFNCSQhLWPZogDiUg 제목: 할머니에게 19금 크리스마스 선물을 드렸을 때 [박막례 할머니] 카테고리 코미디 조회수 조회수 636,558회 좋아요수 1.5만개 싫어요수 156개 게시일 2017. 12. 10.
202 . 썸네일 https://i.ytimg.com/vi/p0WKTc2vLPA/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAoB9FPGILAm8od_jMf-kw5UHF-4w 제목: (힐링) 박막례의 염병할 호주 골드코스트 여행기2 [박막례 할머니] 카테고리 Yuri Seo 조회수 조회수 365,273회 좋아요수 1만개 싫어요수 48개 게시일 2017. 12.

232 . 썸네일 https://i.ytimg.com/vi/JqZVi2P_EMk/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAHZX6FqEzXDAbifvVVmEYN2imJAw 제목: 희한하게 참외를 닮은 프리지아 메이크업 [박막례 할머니] 카테고리 코미디 조회수 조회수 935,872회 좋아요수 1.4만개 싫어요수 441개 게시일 2017. 7. 2.
233 . 썸네일 https://i.ytimg.com/vi/pZJKoWjlOn8/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDIOFSRJaML7HDUvQn1nhywjADlYA 제목: 할머니가 본 영화 속 이제훈이 눈 앞에 나타난다면? [박막례 할머니] 카테고리 코미디 조회수 조회수 1,179,574회 좋아요수 2.8만개 싫어요수 199개 게시일 2017. 6. 23.
234 . 썸네일 https://i.ytimg.com/vi/b5WM-ZGNJJs/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCcz89F8DGk3VZiRTfatXsIFgXnQw 제목: 박막례의 살 빠지는 댄스교실, 박재범 All I Wanna Do  [박막례 할머니] 카테고리 코미디 조회수 조회수 752,183회 좋아요수 1.4만개 싫어요수 130개 게시일 2017. 6. 15.
235 . 썸네일 https://i.ytimg.com/vi/wh_La3oMKcg/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBCuqas2UMiM0gVrP3Sz31So_5U-A 제목: 피부관리샵에서 화가 난 박막례 손님, 롤플레이 ASMR (skin care role play) [박막례 할머니] 카테고리 캐슈넛 조회수 조회수 1,243,675

265 . 썸네일 https://i.ytimg.com/vi/3zlvl1580wA/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAiQzMkZB8pUiGApT5wUgx1IqVQ0Q 제목: 파스타를 처음 먹어봤어요! [박막례 할머니] 카테고리 코미디 조회수 조회수 1,652,609회 좋아요수 1.9만개 싫어요수 306개 게시일 2017. 3. 1.
266 . 썸네일 https://i.ytimg.com/vi/seQQxOvz8rY/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD5kSOSfT3I4WUXGH93fjtKSsCOQg 제목: 71년만에 생애 첫 요가 도전! [박막례 할머니] 카테고리 코미디 조회수 조회수 524,183회 좋아요수 9천개 싫어요수 60개 게시일 2017. 2. 27.
267 . 썸네일 https://i.ytimg.com/vi/ON1I33m9E2o/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBgc1K8aYc9pb9ZkFBncIvDLHluXw 제목: 할머니의 첫 연기 배우기!  녹차드세요 대사의 늪 [박막례 할머니] 카테고리 엔터테인먼트 조회수 조회수 543,671회 좋아요수 1.1만개 싫어요수 72개 게시일 2017. 2. 22.
268 . 썸네일 https://i.ytimg.com/vi/JuWZQK0BqOE/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCW3HExI0px8vfRorHUl33IZQZJPg 제목: 도대체 할머니의 가방엔 뭐가 들어있을까? [박막례 할머니] 카테고리 엔터테인먼트 조회수 조회수 1,584,715회 좋아요수 2.1만개 싫어요수 220개 게시일 2017. 2. 20.
269 . 썸네일 https

In [6]:
import pandas as pd
insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    }) 
insert_data

,thumbnail,title,category,view_count,likes_count,dislikes_count,upload_date
0,https://i.ytimg.com/vi/1LW33ylk8wM/hqdefault.j...,박막례 간장게장 만들기 [박막례 할머니],코미디,"조회수 612,689회",1.9만개,223개,2020. 1. 5.
1,https://i.ytimg.com/vi/BnmL_KbnStE/hqdefault.j...,핫팩 사세요~ 호~ HOT핫HOT핫팩이 왔어요~❤️ [박막례 할머니],코미디,"조회수 126,426회",7.4천개,31개,2020. 1. 3.
2,https://i.ytimg.com/vi/nbQQauKcHmk/hqdefault.j...,그 연예대상 시상식 브이로그!!! [박막례 할머니],코미디,"조회수 538,442회",2.4만개,141개,2019. 12. 29.
3,https://i.ytimg.com/vi/NrTC1wqiGVQ/hqdefault.j...,크리스마스 홈파티 인 파리 [박막례 할머니],코미디,"조회수 246,666회",1만개,73개,2019. 12. 24.
4,https://i.ytimg.com/vi/8UjKbZQHCBE/hqdefault.j...,"(Eng)장성규, 장동윤 보고 온 썰ㅋㅋ (수영고모 화남) [박막례 할머니]",코미디,"조회수 239,294회",8.7천개,55개,2019. 12. 20.
...,...,...,...,...,...,...,...
267,https://i.ytimg.com/vi/ON1I33m9E2o/hqdefault.j...,할머니의 첫 연기 배우기! 녹차드세요 대사의 늪 [박막례 할머니],엔터테인먼트,"조회수 543,671회",1.1만개,72개,2017. 2. 22.
268,https://i.ytimg.com/vi/JuWZQK0BqOE/hqdefault.j...,도대체 할머니의 가방엔 뭐가 들어있을까? [박막례 할머니],엔터테인먼트,"조회수 1,584,715회",2.1만개,220개,2017. 2. 20.
269,https://i.ytimg.com/vi/xLnSAJ67Tuc/hqdefault.j...,우리 같은 영화 본거 맞나요? 박막례 할머니의 공조 리뷰 스포있음 [박막례 할머니],엔터테인먼트,"조회수 401,057회",7.3천개,45개,2017. 2. 16.
270,https://i.ytimg.com/vi/glRTXA7ott8/hqdefault.j...,박막례 할머니의 라이언 초콜릿 만들기 [박막례 할머니],엔터테인먼트,"조회수 1,590,976회",2.7만개,500개,2017. 2. 13.


In [8]:
insert_data.to_csv("Park.csv",encoding="utf8")

In [ ]:
# 소련여자, 박막례, 최고다윽박, ondo온도, 오킹, 오늘의 하늘,튜브몬,박병창